In [1]:
import numpy as np
import skimage
import utils
import pathlib

In [ ]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    
    hist = np.bincount(im.ravel(), minlength=256).astype(np.float64)
    total = hist.sum()
    if total == 0:
        return 0

    p = hist / total

    bins = np.arange(256, dtype=np.float64)
    omega = np.cumsum(p)                 
    mu = np.cumsum(p * bins)
    mu_T = mu[-1]

    denom = omega * (1.0 - omega)
    with np.errstate(divide='ignore', invalid='ignore'):
        sigma_b2 = (mu_T * omega - mu) ** 2 / denom
        sigma_b2[denom == 0] = 0.0

    threshold = int(np.argmax(sigma_b2))
    return threshold

In [ ]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images/thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed/thumbprint-segmented.png
Reading image: images/rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed/rice-shaded-segmented.png
